In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.sparse import csc_matrix, spdiags
from scipy.sparse.linalg import factorized
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Change the NAME.csv
# This code is in Portuguese (PT/BR)
# This code is not finalized yet.

In [ ]:
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = csc_matrix(np.diff(np.eye(L), 2))
    w = np.ones(L)
    for _ in range(niter):
        W = spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        C = factorized(Z)
        z = C(W.dot(y))
        w = p * (y > z) + (1 - p) * (y < z)
    return z

def rubberband_correction(y, x):
    baseline_points_indices = np.argmin(y, axis=0)
    baseline_points_x = x[baseline_points_indices]
    baseline_points_y = y[baseline_points_indices]
    if len(baseline_points_indices) > 1:
        baseline_fit = np.polyfit(baseline_points_x, baseline_points_y, 1)
        baseline = np.polyval(baseline_fit, x)
    else:
        baseline = np.full_like(x, baseline_points_y[0])
    return y - baseline

def linear_baseline_correction(y):
    return y - y[0] - (y[-1] - y[0]) * (np.arange(len(y)) / (len(y) - 1))

def plot_spectra(wavenumbers, spectra, title="FTIR Spectra"):
    fig, ax = plt.subplots(figsize=(8,5))
    for i in range(spectra.shape[0]):
        ax.plot(wavenumbers, spectra[i, :])
    ax.set_xlabel("Wavenumber (cm⁻¹)")
    ax.set_ylabel("Absorbance (a.u.)")
    ax.set_title(title)
    ax.invert_xaxis()
    ax.grid(True)
    plt.tight_layout()
    plt.show()


try:
    # Load the data skipping the first row (wavenumbers in the first row)
    df = pd.read_csv("NAME.csv", sep=",", header=None, skiprows=[0])
    # Read the wavenumbers from the first row separately
    wavenumbers_df = pd.read_csv("NAME.csv", sep=",", nrows=1, header=None)
    wavenumbers = wavenumbers_df.values[0].astype(float)
    spectra_data = df.values
except Exception as e:
    raise FileNotFoundError("Erro ao carregar NAME.csv: " + str(e))

# Estado do processamento
history = [{"wavenumbers": wavenumbers, "spectra": spectra_data, "label": "Original"}]
current_index = 0


operation_menu = widgets.Dropdown(
    options=[
        ("Selecione uma operação", None),
        ("Corte espectral", "cut"),
        ("Suavização (Savitzky-Golay)", "smooth"),
        ("Correção de baseline", "baseline"),
        ("Normalização", "normalize")
    ],
    description="Operação:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="300px")
)

params_box = widgets.VBox([])  # Parâmetros serão preenchidos dinamicamente
back_button = widgets.Button(description="⏪ Voltar", layout=widgets.Layout(width="100px"))
save_button = widgets.Button(description="💾 Salvar Resultado", layout=widgets.Layout(width="200px"))


def apply_cut(start_wave, end_wave):
    wn, sp = history[current_index]["wavenumbers"], history[current_index]["spectra"]
    cut_indices = np.where((wn >= end_wave) & (wn <= start_wave))[0]
    if len(cut_indices) == 0:
        return wn, sp
    return wn[cut_indices], sp[:, cut_indices]

def apply_smoothing(polyorder, window_length, derivative):
    wn, sp = history[current_index]["wavenumbers"], history[current_index]["spectra"]
    smoothed = savgol_filter(sp, window_length=window_length, polyorder=polyorder, deriv=derivative, axis=1)
    return wn, smoothed

def apply_baseline(option, lam=100, p=0.01):
    wn, sp = history[current_index]["wavenumbers"], history[current_index]["spectra"]
    corrected = np.copy(sp)
    if option == "A":
        for i in range(sp.shape[0]):
            corrected[i, :] = rubberband_correction(sp[i, :], wn)
    elif option == "B":
        for i in range(sp.shape[0]):
            corrected[i, :] = linear_baseline_correction(sp[i, :])
    elif option == "C":
        for i in range(sp.shape[0]):
            corrected[i, :] = sp[i, :] - baseline_als(sp[i, :], lam, p)
    return wn, corrected

def apply_normalization(option):
    wn, sp = history[current_index]["wavenumbers"], history[current_index]["spectra"]
    if option == "A":
        min_vals = np.min(sp, axis=1, keepdims=True)
        max_vals = np.max(sp, axis=1, keepdims=True)
        normed = (sp - min_vals) / (max_vals - min_vals)
    elif option == "B":
        mean_vals = np.mean(sp, axis=1, keepdims=True)
        std_vals = np.std(sp, axis=1, keepdims=True)
        normed = (sp - mean_vals) / std_vals
    return wn, normed


def update_plot():
    output_plot.clear_output(wait=True)
    with output_plot:
        plot_spectra(history[current_index]["wavenumbers"], history[current_index]["spectra"],
                     title=history[current_index]["label"])

def add_history(wn, sp, label):
    global current_index
    history.append({"wavenumbers": wn, "spectra": sp, "label": label})
    current_index = len(history) - 1
    update_plot()

def go_back(_):
    global current_index
    if current_index > 0:
        current_index -= 1
        update_plot()

def save_result(_):
    output_df = pd.DataFrame(history[current_index]["spectra"], columns=history[current_index]["wavenumbers"])
    output_df.to_csv("XA_preprocessado.csv", index=False)
    print("Arquivo salvo como XA_preprocessado.csv")

def on_operation_change(change):
    params_box.children = []
    if change["new"] == "cut":
        start_input = widgets.FloatText(description="Início (cm⁻¹):", value=4000)
        end_input = widgets.FloatText(description="Fim (cm⁻¹):", value=400)
        apply_btn = widgets.Button(description="Aplicar Corte")
        def _apply(_):
            wn, sp = apply_cut(start_input.value, end_input.value)
            add_history(wn, sp, f"Corte {start_input.value}-{end_input.value}")
        apply_btn.on_click(_apply)
        params_box.children = [start_input, end_input, apply_btn]

    elif change["new"] == "smooth":
        poly = widgets.IntText(description="Ordem polinômio:", value=3)
        window = widgets.IntText(description="Janela (ímpar):", value=7)
        deriv = widgets.IntText(description="Derivada:", value=0)
        apply_btn = widgets.Button(description="Aplicar Suavização")
        def _apply(_):
            wn, sp = apply_smoothing(poly.value, window.value, deriv.value)
            add_history(wn, sp, f"Suavização SG (poly={poly.value}, win={window.value})")
        apply_btn.on_click(_apply)
        params_box.children = [poly, window, deriv, apply_btn]

    elif change["new"] == "baseline":
        option = widgets.Dropdown(options=[("Rubber Band", "A"), ("Linear", "B"), ("ALS", "C")],
                                  description="Método:")
        lam = widgets.FloatText(description="Lambda (ALS):", value=100.0)
        p = widgets.FloatText(description="p (ALS):", value=0.01)
        apply_btn = widgets.Button(description="Aplicar Baseline")
        def _apply(_):
            wn, sp = apply_baseline(option.value, lam.value, p.value)
            add_history(wn, sp, f"Baseline {option.label}")
        apply_btn.on_click(_apply)
        params_box.children = [option, lam, p, apply_btn]

    elif change["new"] == "normalize":
        option = widgets.Dropdown(options=[("Min-Max", "A"), ("SNV", "B")], description="Método:")
        apply_btn = widgets.Button(description="Aplicar Normalização")
        def _apply(_):
            wn, sp = apply_normalization(option.value)
            add_history(wn, sp, f"Normalização {option.label}")
        apply_btn.on_click(_apply)
        params_box.children = [option, apply_btn]


operation_menu.observe(on_operation_change, names="value")
back_button.on_click(go_back)
save_button.on_click(save_result)


output_plot = widgets.Output()
left_panel = widgets.VBox([operation_menu, params_box, back_button, save_button])
ui = widgets.HBox([left_panel, output_plot])

update_plot()
display(ui)